In [1]:
import os
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
import re

# Set the OpenAI API key
os.environ['OPENAI_API_KEY'] = "sk-proj-vi58QD74lKpYVlS8jF7NT3BlbkFJ7h6LtRORhb8DNhT5Y2z0"

# User inputs to generate NuSMV models
user_inputs = [
    "Safety: temperature should not exceed 80 degrees"
]

# Define the prompt template
prompt_template = ChatPromptTemplate.from_template(
    """
    You are an assistant that helps generate NuSMV models. Generate NuSMV rules with Linear Temporal Logic for the following user input: {user_input}
    Generate the NuSMV format only, no explanation.
    """
)

# Define the output parser
output_parser = StrOutputParser()
model = ChatOpenAI(model="gpt-4o")

# Define the chain
chain = (
    prompt_template 
    | model 
    | output_parser
)

# Function to filter and append NuSMV rules to a model file
def append_rule_to_model(rule, model_path):
    # Remove the triple quotes and ```nusmv tags
    filtered_rule = rule.replace("```nusmv", "").replace("```", "").strip()
    
    # Extract only the relevant sections
    var_section = re.findall(r'VAR\n(.*?)\nASSIGN', filtered_rule, re.DOTALL)
    assign_section = re.findall(r'ASSIGN\n(.*?)\nLTLSPEC', filtered_rule, re.DOTALL)
    ltl_spec = re.findall(r'LTLSPEC\n(.*)', filtered_rule, re.DOTALL)
    
    with open(model_path, 'a') as file:
        if var_section:
            file.write(f"\nVAR\n{var_section[0].strip()}\n")
        if assign_section:
            file.write(f"\nASSIGN\n{assign_section[0].strip()}\n")
        if ltl_spec:
            file.write(f"\nLTLSPEC\n{ltl_spec[0].strip()}\n")

# Path to the model file
model_path = "model.smv"

# Clear the existing model file or create a new one with only the MODULE, VAR, and ASSIGN headers
with open(model_path, 'w') as file:
    file.write("MODULE main\nVAR\nASSIGN\n")

# Process each user input and append the generated NuSMV rules to the model file
for user_input in user_inputs:
    input_data = {"user_input": user_input}
    result = chain.invoke(input_data)
    append_rule_to_model(result, model_path)

# Output the contents of the model file
with open(model_path, 'r') as file:
    model_contents = file.read()
    print(model_contents)


MODULE main
VAR
ASSIGN



In [2]:
import os
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
import re

# Set the OpenAI API key
os.environ['OPENAI_API_KEY'] = "sk-proj-vi58QD74lKpYVlS8jF7NT3BlbkFJ7h6LtRORhb8DNhT5Y2z0"

# Define the prompt template to generate devices and TAP rule
prompt_template = ChatPromptTemplate.from_template(
    """
    You are a smart home assistant. The user wants to automate the following routine: {user_input}. 
    First, list the devices involved and their state variables in the format: device_name : variable_type;. 
    Then, generate the Trigger-Action Rule for the routine and convert it to Linear Temporal Logic (LTL) for NuSMV model. 
    Use this as an example: G (motion_sensor_active -> F switch_on_Aeotec_Outlet_1).
    Generate the rules in Linear Temporal Logic (LTL) for NuSMV model only. No extra explanation.
    """
)

output_parser = StrOutputParser()
model = ChatOpenAI(model="gpt-4o")

# Define the chain
chain = (
    prompt_template 
    | model 
    | output_parser
)

# Function to generate devices and TAP rule
def generate_devices_and_tap_rule(user_input):
    input_data = {
        "user_input": user_input
    }

    result = chain.invoke(input_data).strip()
    print("LLM Result:\n", result)  # Debug statement to print the result from LLM

    # Use regex to extract devices and TAP rule
    devices_output_match = re.search(r'((?:\w+\s*:\s*\w+;\s*)+)(G\s*\(.*\))', result, re.DOTALL)
    if devices_output_match:
        devices_output = devices_output_match.group(1).strip()
        tap_rule = devices_output_match.group(2).strip()
    else:
        raise ValueError("Failed to extract devices output and TAP rule from the result")

    return devices_output, tap_rule

user_input = "Turn the heat up to maximum when temperature drops below 60"
devices_output, trigger_action_rule = generate_devices_and_tap_rule(user_input)

print("Devices Output:")
print(devices_output)
print("\nTrigger Action Rule:")
print(trigger_action_rule)


LLM Result:
 ```
    temperature_sensor : temperature_value;
    thermostat : heat_level;

    G (temperature_value < 60 -> F heat_level = maximum)
    ```
Devices Output:
temperature_sensor : temperature_value;
    thermostat : heat_level;

Trigger Action Rule:
G (temperature_value < 60 -> F heat_level = maximum)


In [3]:
import re
import subprocess

# Path to the model file
model_path = "model.smv"

# Extract variables from the devices_output
tap_variables = re.findall(r'(\w+)\s*:\s*\w+;', devices_output)

# Read the existing model file
with open(model_path, 'r') as file:
    model_contents = file.readlines()

# Insert the variable declarations into the model file
var_index = model_contents.index("VAR\n") + 1
for var in tap_variables:
    var_declaration = f"  {var} : boolean;\n"
    if var_declaration not in model_contents:
        model_contents.insert(var_index, var_declaration)
        var_index += 1

# Insert the ASSIGN declarations
assign_index = model_contents.index("ASSIGN\n") + 1
for var in tap_variables:
    assign_declaration = f"  init({var}) := FALSE;\n"
    if assign_declaration not in model_contents:
        model_contents.insert(assign_index, assign_declaration)
        assign_index += 1

# Write the updated model file back to disk
with open(model_path, 'w') as file:
    file.writelines(model_contents)

# Clean the TAP rule
cleaned_tap_rule = trigger_action_rule.strip().replace("\n", " ")

# Append the TAP rule to the existing model.smv file
def append_tap_rule_to_model(rule, model_path):
    with open(model_path, 'a') as file:
        file.write(f"\nLTLSPEC {rule}\n")

append_tap_rule_to_model(cleaned_tap_rule, model_path)

# Output the contents of the updated model file
with open(model_path, 'r') as file:
    model_contents = file.read()
    print("Updated NuSMV Model Contents:\n", model_contents)

# Run the NuSMV model file and capture the output
def run_nusmv_model(model_path):
    try:
        result = subprocess.run(['NuSMV', model_path], capture_output=True, text=True)
        print("NuSMV Output:\n", result.stdout)
        if result.stderr:
            print("NuSMV Errors:\n", result.stderr)
    except FileNotFoundError:
        print("NuSMV is not installed or not found in the system PATH.")

# Run the NuSMV model
run_nusmv_model(model_path)


Updated NuSMV Model Contents:
 MODULE main
VAR
  temperature_sensor : boolean;
  thermostat : boolean;
ASSIGN
  init(temperature_sensor) := FALSE;
  init(thermostat) := FALSE;

LTLSPEC G (temperature_value < 60 -> F heat_level = maximum)

NuSMV Output:
 *** This is NuSMV 2.5.4 (compiled on Fri Oct 28 14:13:29 UTC 2011)
*** Enabled addons are: compass 
*** For more information on NuSMV see <http://nusmv.fbk.eu>
*** or email to <nusmv-users@list.fbk.eu>.
*** Please report bugs to <nusmv-users@fbk.eu>

*** Copyright (c) 2010, Fondazione Bruno Kessler

*** This version of NuSMV is linked to the CUDD library version 2.4.1
*** Copyright (c) 1995-2004, Regents of the University of Colorado

*** This version of NuSMV is linked to the MiniSat SAT solver. 
*** See http://www.cs.chalmers.se/Cs/Research/FormalMethods/MiniSat
*** Copyright (c) 2003-2005, Niklas Een, Niklas Sorensson 


NuSMV Errors:
 
TYPE ERROR file model.smv: line 9 : undefined identifier 'temperature_value'

TYPE ERROR file mode